In [ ]:
# オリジナルモジュールのインポート
from lib.introngap import PileUp
from lib.gbkparse import Seq_count

# モジュールのインポート
import itertools
import logomaker
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
# クラスのインスタンス化
gbk = Seq_count()

# gbkファイルの読み込み
gbk.read_gbk('../data/gbk/human_ttn.gb')

In [ ]:
n = 5
# すべてのイントロンの3-prime側のn塩基の配列を取得
right_edges = []
for i in range(gbk.intron_num()):
    right_edges.append(str(gbk.intron_seq(i+1)[-n:]))

# すべてのイントロンの5-prime側のn塩基の配列を取得
left_edges = []
for i in range(gbk.intron_num()):
    left_edges.append(str(gbk.intron_seq(i+1)[:n]))

In [ ]:
n = 5
# すべてのイントロンの3-prime側のn塩基の配列を取得
right_edges = {}
for i in range(gbk.intron_num()):
    right_edges[i+1] = str(gbk.intron_seq(i+1)[-n:])
# すべてのイントロンの5-prime側のn塩基の配列を取得
left_edges = {}
for i in range(gbk.intron_num()):
    left_edges[i+1] = str(gbk.intron_seq(i+1)[:n])

In [ ]:
key_combinations = list(itertools.product(right_edges.keys(), left_edges.keys()))
seq_combinations = list(itertools.product(right_edges.values(), left_edges.values()))

intron_numb_combinations = [f"{i}_{j}" for i, j in key_combinations]
intron_seq_combinations = [f"{i}_{j}" for i, j in seq_combinations]

In [ ]:
# ワンホットエンコーディングを行う関数
def one_hot_encode(seq):
    mapping = {'A': [1, 0, 0, 0], 'T': [0, 1, 0, 0], 'G': [0, 0, 1, 0], 'C': [0, 0, 0, 1]}
    return np.array([mapping[s] for s in seq]).flatten()

# エンコーディングされた配列を準備
encoded_sequences = np.array([one_hot_encode(seq) for seq in intron_seq_combinations])

# 主成分分析（PCA）
pca = PCA(n_components=2)  # 2つの主成分を取得
principal_components = pca.fit_transform(encoded_sequences)

# 主成分をデータフレームに変換
df_pca = pd.DataFrame(data=principal_components, columns=['PC1', 'PC2'])

# 結果をプロット
fig = px.scatter(df_pca, x='PC1', y='PC2', hover_name=intron_seq_combinations)
fig.show()